In [1]:
!pip install --upgrade langchain langchain-community langgraph

!pip install langchain-ollama

!pip install langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.4/151.4 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.4 MB/s eta 0:00:00


In [2]:
from typing import List, Dict


In [3]:
from langgraph.graph import StateGraph, START, END


In [4]:
from langchain_openai.chat_models import ChatOpenAI
# from langchain_ollama.llms import OllamaLLM

In [5]:
#Define STate
class State(Dict):
  messages : List[Dict[str, str]]

In [6]:
llm= ChatOpenAI(api_key = "********************"
)

In [7]:
llm.invoke("Asda")

AIMessage(content='Asda is a British supermarket chain that offers a wide range of products including groceries, clothing, electronics, and homeware. It is one of the largest retailers in the UK and is known for its affordable prices and extensive product selection. Asda also offers online shopping and delivery services, making it convenient for customers to shop from the comfort of their own homes.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 9, 'total_tokens': 82, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-eb88848b-7e9c-4e64-bae2-72b424b9d181-0', usage_metadata={'input_tokens': 9, 'output_tokens': 73, 'total_tokens': 82, 'input_token_detail

In [8]:
# Step 2: Initialize StateGraph
graph_builder = StateGraph(State)

In [9]:
# Define chatbot function
def chatbot(state: State):
    response = llm.invoke(state["messages"])
    state["messages"].append({"role": "assistant", "content": response})  # Treat response as a string
    return {"messages": state["messages"]}

In [10]:
# Add nodes and edges
graph_builder.add_node("chatbot", chatbot)

In [11]:
# Add nodes and edges
graph_builder.add_edge(START, "chatbot")

In [12]:
# Add nodes and edges
graph_builder.add_edge("chatbot", END)

In [13]:
graph = graph_builder.compile()

In [14]:
# Stream updates

In [23]:
def stream_graph_updates(user_input: str):
    state = {"messages": [{"role": "user", "content": user_input}]}
    for event in graph.stream(state):
        for value in event.values():
            print("Assistant:", value["messages"][-1]["content"])

In [16]:
# Run chatbot in a loop
while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break

        stream_graph_updates(user_input)
    except Exception as e:
        print(f"An error occurred: {e}")
        break

User: What is LLM?
Assistant: content='LLM stands for Master of Laws, which is an advanced degree in law that is usually pursued by students who have already completed a JD (Juris Doctor) or an equivalent law degree. The LLM program allows students to specialize in a particular area of law and gain a deeper understanding of specific legal topics.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 12, 'total_tokens': 75, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-8083a677-30cc-489f-b545-6ebd40a2a97d-0' usage_metadata={'input_tokens': 12, 'output_tokens': 63, 'total_tokens': 75, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details